Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [18]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [19]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [20]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [21]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [22]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [23]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,...,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [24]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

In [25]:
import numpy as np

In [26]:
true_false_columns = ['Chips', 'Beef', 'Pico', 'Guac', 'Cheese', 'Fries', 'Sour cream', 'Pork', 
                      'Chicken', 'Shrimp', 'Fish', 'Rice', 'Beans', 'Lettuce',
                      'Tomato', 'Bell peper', 'Carrots', 'Cabbage', 'Sauce', 'Salsa.1', 
                      'Cilantro', 'Onion', 'Taquito', 'Pineapple',
                      'Ham', 'Chile relleno', 'Nopales', 'Lobster', 'Queso', 'Egg', 'Mushroom',
                      'Bacon', 'Sushi', 'Avocado', 'Corn', 'Zucchini']

In [27]:
df[true_false_columns] = df[true_false_columns].replace(('x', 'X', np.NaN), (True, True, False))

In [28]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,...,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,2016-01-18,3.5,4.2,False,6.49,3.0,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1,California,2016-01-24,3.5,3.3,False,5.45,3.5,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2,Carnitas,2016-01-24,NaN,NaN,False,4.85,1.5,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,Asada,2016-01-24,NaN,NaN,False,5.25,2.0,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
4,California,2016-01-27,4.0,3.8,True,6.59,4.0,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,True


In [29]:
df.isnull().sum()

Burrito             0
Date                0
Yelp              334
Google            334
Chips               0
Cost                7
Hunger              3
Mass (g)          399
Density (g/mL)    399
Length            138
Circum            140
Volume            140
Tortilla            0
Temp               20
Meat               14
Fillings            3
Meat:filling        9
Uniformity          2
Salsa              25
Synergy             2
Wrap                3
Unreliable        388
NonSD             414
Beef                0
Pico                0
Guac                0
Cheese              0
Fries               0
Sour cream          0
Pork                0
Chicken             0
Shrimp              0
Fish                0
Rice                0
Beans               0
Lettuce             0
Tomato              0
Bell peper          0
Carrots             0
Cabbage             0
Sauce               0
Salsa.1             0
Cilantro            0
Onion               0
Taquito             0
Pineapple 

In [30]:
df.median()

Yelp                4.000000
Google              4.200000
Cost                6.990000
Hunger              3.500000
Mass (g)          540.000000
Density (g/mL)      0.658099
Length             20.000000
Circum             22.000000
Volume              0.770000
Tortilla            3.500000
Temp                4.000000
Meat                3.800000
Fillings            3.500000
Meat:filling        4.000000
Uniformity          3.500000
Salsa               3.500000
Synergy             3.800000
Wrap                4.000000
Beef                0.000000
Pico                0.000000
Guac                0.000000
Cheese              0.000000
Fries               0.000000
Sour cream          0.000000
Pork                0.000000
Chicken             0.000000
Shrimp              0.000000
Fish                0.000000
Rice                0.000000
Beans               0.000000
Lettuce             0.000000
Tomato              0.000000
Bell peper          0.000000
Carrots             0.000000
Cabbage       

In [32]:
df = df.fillna(df.median())

In [33]:
df['Unreliable'].unique()

array([nan, 'x'], dtype=object)

In [34]:
df['Unreliable'] = df['Unreliable'].replace(('x', np.NaN), (True, False))

In [35]:
df['NonSD'].unique()

array([nan, 'x', 'X'], dtype=object)

In [36]:
df['NonSD'] = df['NonSD'].replace(('x', 'X', np.NaN), (True, True, False))

In [37]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,...,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,2016-01-18,3.5,4.2,False,6.49,3.0,540.0,0.658099,20.0,...,False,False,False,False,False,False,False,False,False,False
1,California,2016-01-24,3.5,3.3,False,5.45,3.5,540.0,0.658099,20.0,...,False,False,False,False,False,False,False,False,False,False
2,Carnitas,2016-01-24,4.0,4.2,False,4.85,1.5,540.0,0.658099,20.0,...,False,False,False,False,False,False,False,False,False,False
3,Asada,2016-01-24,4.0,4.2,False,5.25,2.0,540.0,0.658099,20.0,...,False,False,False,False,False,False,False,False,False,False
4,California,2016-01-27,4.0,3.8,True,6.59,4.0,540.0,0.658099,20.0,...,False,False,False,False,False,False,False,False,False,True


In [38]:
df.isnull().sum()

Burrito           0
Date              0
Yelp              0
Google            0
Chips             0
Cost              0
Hunger            0
Mass (g)          0
Density (g/mL)    0
Length            0
Circum            0
Volume            0
Tortilla          0
Temp              0
Meat              0
Fillings          0
Meat:filling      0
Uniformity        0
Salsa             0
Synergy           0
Wrap              0
Unreliable        0
NonSD             0
Beef              0
Pico              0
Guac              0
Cheese            0
Fries             0
Sour cream        0
Pork              0
Chicken           0
Shrimp            0
Fish              0
Rice              0
Beans             0
Lettuce           0
Tomato            0
Bell peper        0
Carrots           0
Cabbage           0
Sauce             0
Salsa.1           0
Cilantro          0
Onion             0
Taquito           0
Pineapple         0
Ham               0
Chile relleno     0
Nopales           0
Lobster           0


In [40]:
train = df[df['Date'] < '2017-01-01']
validate = df[(df['Date'] >= '2017-01-01') & (df['Date'] < '2018-01-01')]
test = df[df['Date'] >= '2018-01-01']

In [41]:
print(train.shape)
print(validate.shape)
print(test.shape)

(298, 59)
(85, 59)
(38, 59)


In [42]:
target = 'Great'
y_train = train[target]
majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_train)

In [43]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

0.5906040268456376

In [44]:
basline_accuracy = accuracy_score(y_train, y_pred)

In [45]:
y_val = validate[target]

In [46]:
from sklearn.linear_model import LogisticRegression

In [47]:
features = ['Hunger', 'Meat', 'Fillings']

In [48]:
X_train = train[features]
X_val = validate[features]

In [49]:
log_reg = LogisticRegression(solver='lbfgs')
log_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)